# Launching CaT Synthesizer

Requirements:

1- Download data: http://svocats.cab.inta-csic.es/catlib

2- Run CaT_2Layers_6D_Autoencoder.ipynb

3- Run CaT_Synth_Preprocess.ipynb

#### ACKNOWLEDGMENT
This work is based on data from the CaT service developed by the Spanish Virtual Observatory in the framework of the IAU Comission G5 Working Group : Spectral Stellar Libraries.

In [1]:
import pygame.midi
import numpy as np
#import matplotlib.image as mpimg
from music21 import *
from PIL import Image
import os
from pythonosc import udp_client

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.midi.init()
pygame.midi.get_count() #Bluetooth Keyboard #4 => 10 I/O

10

## Importing data

In [3]:
img_root = "/Volumes/WORK/DOCTORADO_UPM/Library/CaT/"

In [5]:
vectors = np.load('latent_CaT.npy')

In [6]:
peaks = np.load('peaks_CaT.npy', allow_pickle = True)

In [7]:
r2 = np.load('r_square_CaT.npy')

In [8]:
spectra_set = np.load('Spectra_CaT.npy')

## Chordification of the CaT latent space vectors 

In [13]:
ansi_chords = (np.ceil(100*vectors)).astype(int)
chords = [[] for _ in ansi_chords]
for i in range(ansi_chords.shape[0]):
    for j in range(ansi_chords.shape[1]):
        chords[i].append(pygame.midi.midi_to_ansi_note(ansi_chords[i][j]))

In [14]:
chords

[['D#6', 'F#1', 'F#1', 'C7', 'A#-1', 'G1'],
 ['D4', 'F#3', 'E1', 'D7', 'C#2', 'C4'],
 ['E6', 'A#1', 'A#1', 'C7', 'C0', 'C2'],
 ['B4', 'F2', 'D#1', 'D7', 'C#0', 'G#4'],
 ['A5', 'B2', 'D2', 'C#7', 'C1', 'C#3'],
 ['A#2', 'F1', 'F#3', 'D#7', 'A-1', 'A4'],
 ['A#5', 'F2', 'D1', 'C#7', 'C#0', 'D#3'],
 ['G#5', 'G3', 'D1', 'C#7', 'E0', 'F#3'],
 ['F5', 'D2', 'D#0', 'D7', 'B-1', 'D#3'],
 ['A#6', 'D#6', 'C7', 'C1', 'C#-1', 'D-1'],
 ['G3', 'F#1', 'F5', 'C#7', 'A#0', 'G2'],
 ['A#2', 'C3', 'B3', 'D7', 'F2', 'E4'],
 ['C5', 'A#0', 'F#0', 'D7', 'D2', 'A1'],
 ['A#4', 'F2', 'A0', 'C#7', 'C2', 'B1'],
 ['D#5', 'B1', 'D#0', 'D7', 'F#0', 'G3'],
 ['E4', 'E6', 'C#2', 'C7', 'C0', 'D#5'],
 ['F5', 'E4', 'F1', 'D7', 'G0', 'E5'],
 ['B3', 'G#2', 'C#7', 'F6', 'E0', 'A#0'],
 ['G5', 'C#2', 'A#2', 'C#7', 'D1', 'E2'],
 ['C#5', 'G#3', 'C1', 'D7', 'A-1', 'D#5'],
 ['F#5', 'E3', 'G#1', 'D7', 'A#0', 'B3'],
 ['C#2', 'G#5', 'B3', 'C#7', 'E2', 'A#5'],
 ['A#5', 'F#3', 'D#1', 'C#7', 'F#1', 'G#3'],
 ['F#5', 'E5', 'C#1', 'C#7', 'E0',

## Running

In [ ]:
message = pygame.midi.Input(4) 
active_notes = 0
player0 = pygame.midi.Output(5)
player1 = pygame.midi.Output(6)
player2 = pygame.midi.Output(7)
player3 = pygame.midi.Output(8)

while True:
    if message.poll() and active_notes == 0:
        input_message = message.read(1)[0][0][1]
        
        print("input: ", input_message)
        note_on = pygame.midi.midi_to_ansi_note(input_message)
        print("note on: ", note_on)
        active_notes = 1
        
        chord_ref = [note_on]

        chord1 = chord.Chord(chord_ref)

        for i in range(len(chords)):
            try:
                chord2 = chord.Chord(chords[i])
                if (chord1.root () == chord2.root ()):
                    print("Play Spectrum:", spectra_set[i])
                    
                    spectrum_fits = spectra_set[i]
                    img_name = os.path.splitext(spectrum_fits)[0]
                    new_extension = '.png'
                    img_png = img_root + img_name + new_extension    
                    image = Image.open(img_png)
                    # Saving the image to open in Cabbage
                    image.save('Spectrum.png')
                    image.close()
                    
                    print("Ansi latent vector: ", ansi_chords[i])
                    print("Chord: ", chord2.commonName)
                    print("Number of lines: ", len(peaks[i]))
                    print("Absorption wavelengths (Angstrom): ", peaks[i])
                    print("Rsquare: ", r2[i])
                    
                    client_s = udp_client.SimpleUDPClient("127.0.0.1", 9989)
                    client_s.send_message("/s", 1)
                    
                    client0 = udp_client.SimpleUDPClient("127.0.0.1", 9990)   
                    client0.send_message("/r2", r2[i])
                    
                    client1 = udp_client.SimpleUDPClient("127.0.0.1", 9991)   
                    client1.send_message("/lines", len(peaks[i]))             
                 
                    #playing notes
                                        
                    player0.note_on(ansi_chords[i][0], 25)
                    player0.note_on(ansi_chords[i][1], 25)
                    player0.note_on(ansi_chords[i][2], 25)
                    player0.note_on(ansi_chords[i][3], 25)
                    player0.note_on(ansi_chords[i][4], 25)
                    player0.note_on(ansi_chords[i][5], 25)

                    break    
            
            except:
                i += 1
            
        
    if message.poll() and active_notes == 1:
        input_message = message.read(1)[0][0][1]
        print("note off")
        print("---------------------------------")
        
        active_notes = 0
        client_s = udp_client.SimpleUDPClient("127.0.0.1", 9989)
        client_s.send_message("/s", 0)     
        client1.send_message("/lines", 0)

        player0.note_off(ansi_chords[i][0], 25)
        player0.note_off(ansi_chords[i][1], 25)
        player0.note_off(ansi_chords[i][2], 25)
        player0.note_off(ansi_chords[i][3], 25)
        player0.note_off(ansi_chords[i][4], 25)
        player0.note_off(ansi_chords[i][5], 25)


input:  81
note on:  A5
Play Spectrum: scan332.fits
Ansi latent vector:  [81 44 26 97 20 40]
Chord:  Balinese Pelog pentatonic
Number of lines:  8
Absorption wavelengths (Angstrom):  [ 40 101 175 227 369 539 726 786]
Rsquare:  0.9465337840053326
note off
---------------------------------
input:  81
note on:  A5
Play Spectrum: scan332.fits
Ansi latent vector:  [81 44 26 97 20 40]
Chord:  Balinese Pelog pentatonic
Number of lines:  8
Absorption wavelengths (Angstrom):  [ 40 101 175 227 369 539 726 786]
Rsquare:  0.9465337840053326
note off
---------------------------------
input:  86
note on:  D6
Play Spectrum: scan626.fits
Ansi latent vector:  [56 44 86 87 90 24]
Chord:  augmented-sixth pentachord
Number of lines:  8
Absorption wavelengths (Angstrom):  [115 181 231 293 372 472 605 787]
Rsquare:  0.9483380038875411
note off
---------------------------------
input:  84
note on:  C6
Play Spectrum: scan647.fits
Ansi latent vector:  [84 44 47 96 16 47]
Chord:  augmented major tetrachord
Numb

note off
---------------------------------
input:  86
note on:  D6
Play Spectrum: scan626.fits
Ansi latent vector:  [56 44 86 87 90 24]
Chord:  augmented-sixth pentachord
Number of lines:  8
Absorption wavelengths (Angstrom):  [115 181 231 293 372 472 605 787]
Rsquare:  0.9483380038875411
note off
---------------------------------
input:  84
note on:  C6
Play Spectrum: scan647.fits
Ansi latent vector:  [84 44 47 96 16 47]
Chord:  augmented major tetrachord
Number of lines:  4
Absorption wavelengths (Angstrom):  [176 227 369 539]
Rsquare:  0.9672903447580554
note off
---------------------------------
input:  88
note on:  E6
Play Spectrum: scan249.fits
Ansi latent vector:  [63 75 65 91 88 52]
Chord:  major-second tetracluster
Number of lines:  5
Absorption wavelengths (Angstrom):  [230 290 370 472 609]
Rsquare:  0.9234709651911354
note off
---------------------------------
input:  88
note on:  E6
Play Spectrum: scan249.fits
Ansi latent vector:  [63 75 65 91 88 52]
Chord:  major-second te

input:  90
note on:  F#6
Play Spectrum: scan163.fits
Ansi latent vector:  [59 70 61 90 91 44]
Chord:  forte class 6-11A
Number of lines:  8
Absorption wavelengths (Angstrom):  [176 228 294 369 472 605 738 785]
Rsquare:  0.8527578656407343
note off
---------------------------------
input:  87
note on:  D#6
Play Spectrum: scan369.fits
Ansi latent vector:  [65 73 64 87 94 44]
Chord:  blues scale
Number of lines:  8
Absorption wavelengths (Angstrom):  [114 181 227 294 372 472 604 784]
Rsquare:  0.9210117662900711
note off
---------------------------------
input:  85
note on:  C#6
note off
---------------------------------
input:  85
note on:  C#6
note off
---------------------------------
input:  84
note on:  C6
Play Spectrum: scan647.fits
Ansi latent vector:  [84 44 47 96 16 47]
Chord:  augmented major tetrachord
Number of lines:  4
Absorption wavelengths (Angstrom):  [176 227 369 539]
Rsquare:  0.9672903447580554
note off
---------------------------------
input:  83
note on:  B5
Play Spe

note off
---------------------------------
input:  80
note on:  G#5
Play Spectrum: scan088.fits
Ansi latent vector:  [80 46 29 98 14 59]
Chord:  flat-ninth pentachord
Number of lines:  7
Absorption wavelengths (Angstrom):  [ 40 101 175 227 368 539 757]
Rsquare:  0.8992004394510826
note off
---------------------------------
input:  53
note on:  F3
Play Spectrum: scan335.fits
Ansi latent vector:  [53 56 67 80 97 22]
Chord:  diminished-major ninth chord
Number of lines:  9
Absorption wavelengths (Angstrom):  [ 75 115 181 231 293 372 473 605 786]
Rsquare:  0.9677178905703709
note off
---------------------------------
input:  53
note on:  F3
Play Spectrum: scan335.fits
Ansi latent vector:  [53 56 67 80 97 22]
Chord:  diminished-major ninth chord
Number of lines:  9
Absorption wavelengths (Angstrom):  [ 75 115 181 231 293 372 473 605 786]
Rsquare:  0.9677178905703709
note off
---------------------------------
input:  50
note on:  D3
Play Spectrum: scan579.fits
Ansi latent vector:  [78 50 27 

input:  96
note on:  C7
Play Spectrum: scan048.fits
Ansi latent vector:  [56 96 44 84 72 36]
Chord:  Diminished Fourth with octave doublings
Number of lines:  13
Absorption wavelengths (Angstrom):  [ 77 135 183 235 294 367 422 472 544 603 671 732 784]
Rsquare:  0.6445895003390176
note off
---------------------------------
input:  96
note on:  C7
Play Spectrum: scan048.fits
Ansi latent vector:  [56 96 44 84 72 36]
Chord:  Diminished Fourth with octave doublings
Number of lines:  13
Absorption wavelengths (Angstrom):  [ 77 135 183 235 294 367 422 472 544 603 671 732 784]
Rsquare:  0.6445895003390176
note off
---------------------------------
input:  84
note on:  C6
Play Spectrum: scan647.fits
Ansi latent vector:  [84 44 47 96 16 47]
Chord:  augmented major tetrachord
Number of lines:  4
Absorption wavelengths (Angstrom):  [176 227 369 539]
Rsquare:  0.9672903447580554
note off
---------------------------------
input:  88
note on:  E6
Play Spectrum: scan249.fits
Ansi latent vector:  [63 7

...

Ansi latent vector:  [50 54 56 98 38 65]
Chord:  major-third diminished tetrachord
Number of lines:  4
Absorption wavelengths (Angstrom):  [175 227 369 784]
Rsquare:  0.9212207100895606
note off
---------------------------------
input:  58
note on:  A#3
Play Spectrum: scan256.fits
Ansi latent vector:  [58 46 82 82 94 19]
Chord:  Augmented Second with octave doublings
Number of lines:  10
Absorption wavelengths (Angstrom):  [ 30  75 115 181 231 294 372 473 605 784]
Rsquare:  0.9495474862357018
note off
---------------------------------
input:  58
note on:  A#3
Play Spectrum: scan256.fits
Ansi latent vector:  [58 46 82 82 94 19]
Chord:  Augmented Second with octave doublings
Number of lines:  10
Absorption wavelengths (Angstrom):  [ 30  75 115 181 231 294 372 473 605 784]
Rsquare:  0.9495474862357018
note off
---------------------------------
input:  59
note on:  B3
Play Spectrum: scan368.fits
Ansi latent vector:  [46 48 59 98 41 64]
Chord:  forte class 6-12A
Number of lines:  5
Absorpti

input:  84
note on:  C6
Play Spectrum: scan647.fits
Ansi latent vector:  [84 44 47 96 16 47]
Chord:  augmented major tetrachord
Number of lines:  4
Absorption wavelengths (Angstrom):  [176 227 369 539]
Rsquare:  0.9672903447580554
note off
---------------------------------
input:  83
note on:  B5
Play Spectrum: scan217.fits
Ansi latent vector:  [61 37 83 77 95 15]
Chord:  whole-tone tetramirror
Number of lines:  9
Absorption wavelengths (Angstrom):  [ 51 115 181 231 293 372 473 605 786]
Rsquare:  0.9487035084849577
note off
---------------------------------
input:  84
note on:  C6
Play Spectrum: scan647.fits
Ansi latent vector:  [84 44 47 96 16 47]
Chord:  augmented major tetrachord
Number of lines:  4
Absorption wavelengths (Angstrom):  [176 227 369 539]
Rsquare:  0.9672903447580554
note off
---------------------------------
input:  83
note on:  B5
Play Spectrum: scan217.fits
Ansi latent vector:  [61 37 83 77 95 15]
Chord:  whole-tone tetramirror
Number of lines:  9
Absorption wavelen

note off
---------------------------------
input:  65
note on:  F4
Play Spectrum: scan654.fits
Ansi latent vector:  [73 65 45 97 31 66]
Chord:  augmented pentacluster
Number of lines:  3
Absorption wavelengths (Angstrom):  [175 227 369]
Rsquare:  0.9267398997852
note off
---------------------------------
input:  68
note on:  G#4
Play Spectrum: scan153.fits
Ansi latent vector:  [71 41 27 98 13 68]
Chord:  combinatorial RI (RI6)
Number of lines:  6
Absorption wavelengths (Angstrom):  [101 175 227 368 538 757]
Rsquare:  0.9569471149104373
note off
---------------------------------
input:  54
note on:  F#3
Play Spectrum: scan260.fits
Ansi latent vector:  [54 64 56 94 88 52]
Chord:  whole-tone tetramirror
Number of lines:  6
Absorption wavelengths (Angstrom):  [230 295 371 472 605 784]
Rsquare:  0.9293345731388687
note off
---------------------------------
input:  73
note on:  C#5
Play Spectrum: scan688.fits
Ansi latent vector:  [73 39 52 98 16 62]
Chord:  chromatic tetramirror
Number of li

note off
---------------------------------
input:  94
note on:  A#6
Play Spectrum: scan276.fits
Ansi latent vector:  [85 47 20 94 67 28]
Chord:  combinatorial I (I11)
Number of lines:  9
Absorption wavelengths (Angstrom):  [175 227 293 369 473 604 681 733 786]
Rsquare:  0.9079899837436891
note off
---------------------------------
input:  94
note on:  A#6
Play Spectrum: scan276.fits
Ansi latent vector:  [85 47 20 94 67 28]
Chord:  combinatorial I (I11)
Number of lines:  9
Absorption wavelengths (Angstrom):  [175 227 293 369 473 604 681 733 786]
Rsquare:  0.9079899837436891
note off
---------------------------------
input:  94
note on:  A#6
Play Spectrum: scan276.fits
Ansi latent vector:  [85 47 20 94 67 28]
Chord:  combinatorial I (I11)
Number of lines:  9
Absorption wavelengths (Angstrom):  [175 227 293 369 473 604 681 733 786]
Rsquare:  0.9079899837436891
note off
---------------------------------
input:  82
note on:  A#5
Play Spectrum: scan283.fits
Ansi latent vector:  [37 80 59 97 

note off
---------------------------------
input:  84
note on:  C6
Play Spectrum: scan647.fits
Ansi latent vector:  [84 44 47 96 16 47]
Chord:  augmented major tetrachord
Number of lines:  4
Absorption wavelengths (Angstrom):  [176 227 369 539]
Rsquare:  0.9672903447580554
note off
---------------------------------
input:  83
note on:  B5
Play Spectrum: scan217.fits
Ansi latent vector:  [61 37 83 77 95 15]
Chord:  whole-tone tetramirror
Number of lines:  9
Absorption wavelengths (Angstrom):  [ 51 115 181 231 293 372 473 605 786]
Rsquare:  0.9487035084849577
note off
---------------------------------
input:  86
note on:  D6
Play Spectrum: scan626.fits
Ansi latent vector:  [56 44 86 87 90 24]
Chord:  augmented-sixth pentachord
Number of lines:  8
Absorption wavelengths (Angstrom):  [115 181 231 293 372 472 605 787]
Rsquare:  0.9483380038875411
note off
---------------------------------
input:  81
note on:  A5
Play Spectrum: scan332.fits
Ansi latent vector:  [81 44 26 97 20 40]
Chord:  Ba